In [1]:
import numpy as np
import pandas as pd
import glob
import xarray as xr
import netCDF4 
from tqdm import tqdm_notebook as tqdm
import os
from datetime import date
import datetime

Notes:

Time since 2020-1-1

https://github.com/eurec4a/meta/blob/master/naming_conventions.md

https://cfconventions.org/cf-conventions/cf-conventions.html#time-coordinate


In [2]:
#Load BAHAMAS files (version from DLR-HALO-DATABASE)
file_list = sorted(glob.glob("/Users/marcusklingebiel/Desktop/BAHAMAS/"+"*.nc"))

prefix = "EUREC4A_HALO_BAHAMAS_PositionAttitude_"

In [3]:
file_list[0][-12:-4]

'20200115'

In [18]:
#Delete all nc files 
os.system("rm *.nc")

#Load old file
for i in tqdm(range(len(file_list))):
#for i in range(0,1):
    file = xr.open_dataset(file_list[i],decode_times=False)
    flight_acronym = file_list[i][-12:-4]
    flight_id = "HALO-"+file_list[i][-8:-4]
    filename = file_list[i][-66:]

    #Create new netCDF file
    ncfile = netCDF4.Dataset(prefix+flight_acronym+'_v1.0.nc',mode='w',format='NETCDF4_CLASSIC')

    #Create attributes
    ncfile.title='EUREC4A position and attitude data'
    ncfile.subtitle="10Hz subset based on DLR BAHAMAS data, processed by A.Giez, V.Dreiling, M.Zoeger, Ch. Mallaun; email: andreas.giez@dlr.de"
    ncfile.mission="EUREC4A"
    ncfile.platform="HALO"
    ncfile.instrument = "BAHAMAS"
    ncfile.flight_id=str(flight_id)
    ncfile.source_file=str(filename)
    ncfile.date_last_revised=file.date_last_revised
    ncfile.featureType = "trajectory"
    ncfile.Conventions = "CF-1.7"
    today = date.today()
    ncfile.version = "1.0"
    ncfile.history = "acquired by HALO BAHAMAS, processed and quality checked by DLR FX, subsetted and reformatted by Marcus Klingebiel"
    ncfile.file_created = "File created by M. Klingebiel (email: marcus.klingebiel@uni-leipzig.de) on "+today.strftime("%B %d, %Y")
    

    #Create dimension
    dim_dim = ncfile.createDimension('time', len(file.TIME.values[:]))

    #Create variables
    time = ncfile.createVariable('time', np.float64, ('time',),fill_value=-9900.0)
    time.units = "seconds since 2020-1-1 00:00"
    time.long_name = 'Time in seconds since 2020-1-1'
    time.standard_name = "time"
    time.calendar = "standard"
    time.axis = "T"
    
    #timeold = ncfile.createVariable('timeOld', np.float64, ('time',),fill_value=-9900.0)
    #timeold.units = "seconds since 1970-1-1"
    #timeold.long_name = 'time in seconds since 1970-1-1 (UTC)'
    
    lat = ncfile.createVariable('lat', np.float64, ('time',),fill_value=-9900.0)
    lat.units = 'degrees_north'
    lat.long_name = 'WGS84 Datum/Latitude'
    lat.standard_name = "latitude"
    lat.axis = "Y"

    lon = ncfile.createVariable('lon', np.float64, ('time',),fill_value=-9900.0)
    lon.units = 'degrees_east'
    lon.long_name = 'WGS84 Datum/Longitude'
    lon.standard_name = "longitude"
    lon.axis = "X"

    alt = ncfile.createVariable('alt', np.float32, ('time',),fill_value=-9900.0)
    alt.units = 'm'
    alt.positive = "up"
    alt.long_name = 'WGS84 Datum/Elliptical Height'
    alt.standard_name = "height_above_reference_ellipsoid"
    alt.axis = "Z"

    roll = ncfile.createVariable('roll', np.float32, ('time',),fill_value=-9900.0)
    roll.units = 'degrees'
    roll.long_name = 'roll angle'
    roll.standard_name = 'platform_roll_starboard_down'
    roll.coordinates = "lat lon alt"

    pitch = ncfile.createVariable('pitch', np.float32, ('time',),fill_value=-9900.0)
    pitch.units = 'degrees'
    pitch.long_name = 'pitch angle'
    pitch.standard_name = "platform_pitch_fore_up"
    roll.coordinates = "lat lon alt"

    heading = ncfile.createVariable('heading', np.float32, ('time',),fill_value=-9900.0)
    heading.units = 'degrees'
    heading.long_name = 'true heading'
    heading.standard_name = 'platform_yaw_fore_starboard'
    roll.coordinates = "lat lon alt"
    
    tas = ncfile.createVariable('tas', np.float32, ('time',),fill_value=-9900.0)
    tas.units = 'm/s'
    tas.long_name = 'Calculated True Airspeed'
    tas.standard_name = "platform_speed_wrt_air"
    roll.coordinates = "lat lon alt"
    
    #Write variables
    #timeold[:]    = file.TIME.values[:]
    time[:]= file.TIME.values[:] - np.float64(datetime.datetime(2020,1,1,0,0).strftime("%s"))
    lat[:]     = file.IRS_LAT.values[:]
    lon[:]     = file.IRS_LON.values[:]
    alt[:]     = file.IRS_ALT.values[:]
    roll[:]    = file.IRS_PHI.values[:]
    pitch[:]   = file.IRS_THE.values[:]
    heading[:] = file.IRS_HDG.values[:]
    tas[:]     = file.TAS.values[:]


    #Close file
    ncfile.close()




/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


In [5]:
test = xr.load_dataset(prefix+flight_acronym+'_v1.0.nc')

In [6]:
test.time.values

array(['2020-02-18T11:11:05.000000000', '2020-02-18T11:11:05.099999904',
       '2020-02-18T11:11:05.200000047', ...,
       '2020-02-18T19:55:31.700000047', '2020-02-18T19:55:31.799999952',
       '2020-02-18T19:55:31.900000095'], dtype='datetime64[ns]')